In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2
#%matplotlib inline

In [2]:
from xopt import nsga2, creator
from xopt.cnsga import *

import array
import time
from deap import base, tools,  algorithms
from deap.benchmarks.tools import diversity, convergence, hypervolume
import numpy as np

import random

# Test Problems

In [3]:
NAME = 'TNK'
BOUND_LOW, BOUND_UP = [0.0, 0.0], [3.14159, 3.14159]  


X_RANGE = [0, 1.4]
Y_RANGE = [0, 1.4]

# Pure number version
def TNK(individual):    
    x1=individual[0]
    x2=individual[1]
    objectives =  (x1, x2)
    constraints = (x1**2+x2**2-1.0 - 0.1*np.cos(16*np.arctan2(x1, x2)), 0.5-(x1-0.5)**2-(x2-0.5)**2 )
    return objectives, constraints

# labeled version
def evaluate_TNK(inputs):
    
    info = {'some':'info', 'about':['the', 'run']}
    ind = [inputs['x1'], inputs['x2']]
    objectives, constraints = TNK(ind)    
    outputs = {'y1':objectives[0], 'y2':objectives[1], 'c1':constraints[0], 'c2':constraints[1]}
    
    return outputs

VOCS = {
    'name':'TNK_test',
    
    'variables': {
        'x1':[0, 3.14159],
        'x2':[0, 3.14159]
    },
    'objectives':{
        'y1':'MINIMIZE',
        'y2':'MINIMIZE'
        
    },
    'constraints':{
        'c1': ['GREATER_THAN', 0],
        'c2': ['GREATER_THAN', 0]
        
    },
    'constants':{'a':'dummy_constant'},
    'linked_variables':{'x9':'x1'}
    
    
}





# Setup Toolbox

In [4]:
toolbox = cnsga_toolbox(VOCS)

In [5]:
# Register test proglem as toolbox.evaluate
toolbox.register('evaluate', TNK)

# Test

In [6]:
toolbox.population(n=10)

[Individual('d', [2.682834074901414, 1.5784687990818147]),
 Individual('d', [1.1533549141735688, 1.3207016229425943]),
 Individual('d', [0.849702440590457, 0.2086299525541964]),
 Individual('d', [2.189664560769876, 0.06706864798819603]),
 Individual('d', [3.088382160188962, 1.8155199590671665]),
 Individual('d', [1.2183345993279426, 0.44763894251487046]),
 Individual('d', [1.9349985980434588, 1.4103689524681138]),
 Individual('d', [2.3138338859847623, 2.9319356653098607]),
 Individual('d', [1.131598266639435, 1.4153683076674497]),
 Individual('d', [2.536957219564738, 3.0843345071931494])]

In [7]:
creator.Individual([1,2,3])

Individual('d', [1.0, 2.0, 3.0])

In [ ]:
toolbox.evaluate(
    creator.Individual([1,2,3])
)

# Parallel method

In [8]:
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor
from concurrent.futures import ThreadPoolExecutor as PoolExecutor

# Continuous NSGA-II, -III Loop

In [25]:
#--------------------------------------------    
#--------------------------------------------

def nsga(
          vocs=None,
          population=None,
          toolbox=None,
          seed=None,
          evaluate_f=None,
          output_path=None,
          max_generations = 2,
          population_size = 4,
          crossover_probability = 0.9,
          mutation_probability = 1.0,
          selection='auto',
          verbose=True):
    """  
    NSGA-II, NSGA-III
    
    Requires either a DEAP toolbox or a vocs dict. 
    
    If an output_path is given, regular outputs:
        gen_{i}.json
        pop_{i}.json
    will be written for each generation i, and the best population at that generation.
    These files can be used for restarting the function. 
    
    
    """
    random.seed(seed)
    MU = population_size
    CXPB = crossover_probability
    MUTPB = mutation_probability
    

    # Initial population

    # Verbose print helper
    def vprint(*a, **k):
        if verbose:
            print(*a, **k)
            sys.stdout.flush()
    
    vprint(cnsga_logo)
    
    # Setup saving to file
    if output_path:
        path = full_path(output_path)
        assert os.path.exists(path), f'output_path does not exist {path}'
        
        def save(pop, prefix, generation):
            file = f'{prefix}{generation}.json'
            data = pop_to_data(vocs, pop, generation=generation)
            fullfile = os.path.join(path, file)
            with open(fullfile, 'w') as f:
                json.dump(data, f, ensure_ascii=True, indent=4)
            vprint('Pop written to', fullfile)
        
    else:
        # Dummy save
        def save(pop, prefix, generation):
            pass
        
    # Toolbox
    if not toolbox:
        vprint('Creating toolbox from vocs.')
        toolbox = cnsga_toolbox(vocs, selection=selection, verbose=verbose)
        toolbox.register('evaluate', cnsga_evaluate, evaluate_f=evaluate_f, vocs=vocs)
        if verbose:
            print('vocs:')
            pprint(vocs) # Pretty print dict
        
    # Initial pop
    if population:
        pop = pop_init(vocs, population['variables'])
        if 'generation' in population:
            generation = population['generation']+1
        else:
            generation=0
        MU = len(pop)
        vprint(f'Initializing with existing population, size {MU}')
    else:
        generation = 0
        #pop = toolbox.population(n=MU)   
        pop = pop_init_random(vocs, n=MU)
        vprint(f'Initializing with a new population, size {MU}')
    assert MU % 4 == 0, f'Population size (here {MU}) must be a multiple of 4'        
        
    
    
    t0 = time.time()
    
    new_offspring = []
    for i in range(3):
        generation += 1
        
        pop = toolbox.select(pop + new_offspring, MU)
        
        vecs = get_vecs(algorithms.varAnd(pop, toolbox, CXPB, MUTPB))
        futures = map(toolbox.evaluate, vecs) 
        new_offspring = [form_ind(res) for res in futures]
        
        
        
        #save(new_offspring, 'gen_', generation)
        #save(pop, 'pop_', generation)
        t1 = time.time()
        dt = t1-t0
        t0 = t1
        vprint(f'Generation {generation} completed in {dt/60:0.5f} minutes')        
            
    final_population = pop_to_data(vocs, pop, generation=generation)
            
    return final_population


def evolve(pop, toolbox, max_generations):
        
        pop = toolbox.select(pop + new_offspring, MU)
        
        vecs = get_vecs(algorithms.varAnd(pop, toolbox, CXPB, MUTPB))
        futures = map(toolbox.evaluate, vecs) 
        new_offspring = [form_ind(res) for res in futures]
        
        
        
        #save(new_offspring, 'gen_', generation)
        #save(pop, 'pop_', generation)


#toolbox.register('evaluate', EVALUATE)
executor = PoolExecutor()

pop =  nsga(vocs=VOCS, evaluate_f=evaluate_TNK) 




 ▄████▄   ███▄    █   ██████   ▄████  ▄▄▄      
▒██▀ ▀█   ██ ▀█   █ ▒██    ▒  ██▒ ▀█▒▒████▄    
▒▓█    ▄ ▓██  ▀█ ██▒░ ▓██▄   ▒██░▄▄▄░▒██  ▀█▄  
▒▓▓▄ ▄██▒▓██▒  ▐▌██▒  ▒   ██▒░▓█  ██▓░██▄▄▄▄██ 
▒ ▓███▀ ░▒██░   ▓██░▒██████▒▒░▒▓███▀▒ ▓█   ▓██▒
░ ░▒ ▒  ░░ ▒░   ▒ ▒ ▒ ▒▓▒ ▒ ░ ░▒   ▒  ▒▒   ▓▒█░
  ░  ▒   ░ ░░   ░ ▒░░ ░▒  ░ ░  ░   ░   ▒   ▒▒ ░
░           ░   ░ ░ ░  ░  ░  ░ ░   ░   ░   ▒   
░ ░               ░       ░        ░       ░  ░
░                                              


Continous Non-dominated Sorting Genetic Algorithm


Creating toolbox from vocs.
Warning in cnsga_toolbox: Redefining creator.Individual
Created toolbox with 2 variables, 2 constraints, and 2 objectives.
    Using selection algorithm: nsga2
vocs:
{'constants': {'a': 'dummy_constant'},
 'constraints': {'c1': ['GREATER_THAN', 0], 'c2': ['GREATER_THAN', 0]},
 'linked_variables': {'x9': 'x1'},
 'name': 'TNK_test',
 'objectives': {'y1': 'MINIMIZE', 'y2': 'MINIMIZE'},
 'variables': {'x1': [0, 3.14159], 'x2': [0, 3.14

In [23]:
def f(x):
    return 2*x
fut = map(f, [1,2])
[res.result() for res in fut]

AttributeError: 'int' object has no attribute 'result'

# Parallel method

In [ ]:
# Dask distributed

from dask.distributed import Client
client = Client()
client

In [ ]:
#toolbox.register('evaluate', EVALUATE)
pop = cnsga(client, vocs=C, max_generations = 40, population_size=64) 

In [ ]:
#pop = cnsga(client, toolbox=toolbox, evaluate_f=F) 

# Recreate plots in Deb paper

In [ ]:
import matplotlib.pyplot as plt

def plot_pop(pop):
    fig, ax = plt.subplots(figsize=(5,5))

    front = np.array([ind.fitness.values for ind in pop])
    ax.scatter(front[:,0], front[:,1], color='blue')
    ax.set_xlim(X_RANGE)
    ax.set_ylim(Y_RANGE)
    ax.set_aspect('auto')
    ax.set_title(NAME)
#plot_pop(pop)

# MPI

In [ ]:
from mpi4py.futures import MPIPoolExecutor as PoolExecutor

In [ ]:
from xopt import vocs_tools
from xopt.cnsga import cnsga as xcnsga
VOCS = vocs_tools.load_vocs('TNK_test.json')

In [ ]:
if __name__ == '__main__':
    with PoolExecutor() as executor:
        #pop = cnsga(executor, toolbox=toolbox, evaluate_f=EVALUATE) 
        
        pop=xcnsga(executor, vocs=VOCS, evaluate_f=evaluate_TNK, max_generations=10, population_size=40)
        plot_pop(pop)

In [ ]:
#!jupyter nbconvert --to script cnsga.ipynb